In [2]:
from openai import OpenAI
from mem0 import Memory

# Set the OpenAI API key

class CustomerSupportAIAgent:
    def __init__(self):
        """
        Initialize the CustomerSupportAIAgent with memory configuration and OpenAI client.
        """
        config = {
            "vector_store": {
                "provider": "qdrant",
                "config": {
                    "host": "localhost",
                    "port": 6333,
                }
            },
            'version':'v1.1'
        }
        self.memory = Memory.from_config(config)
        self.client = OpenAI()
        self.app_id = "customer-support"

    def handle_query(self, query, user_id=None):
        """
        Handle a customer query and store the relevant information in memory.

        :param query: The customer query to handle.
        :param user_id: Optional user ID to associate with the memory.
        """
        # Start a streaming chat completion request to the AI
        stream = self.client.chat.completions.create(
            model="gpt-4",
            stream=True,
            messages=[
                {"role": "system", "content": "You are a customer support AI agent."},
                {"role": "user", "content": query}
            ]
        )
        # Store the query in memory
        self.memory.add(query, user_id=user_id, metadata={"app_id": self.app_id})

        # Print the response from the AI in real-time
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                print(chunk.choices[0].delta.content, end="")

    def get_memories(self, user_id=None):
        """
        Retrieve all memories associated with the given customer ID.

        :param user_id: Optional user ID to filter memories.
        :return: List of memories.
        """
        return self.memory.get_all(user_id=user_id)

# Instantiate the CustomerSupportAIAgent
support_agent = CustomerSupportAIAgent()

# Define a customer ID
customer_id = "jane_doe"

# Handle a customer query
support_agent.handle_query("I need help with my recent order. It hasn't arrived yet.", user_id=customer_id)


I'm sorry to hear that your order hasn't arrived yet. I'd be happy to assist you with this. Can you please provide me with the order number or any other information that could help me find your order in our system?

In [10]:
support_agent.handle_query("the order number 678.", user_id=customer_id)


I'm sorry, but I need a bit more context in order to assist you efficiently. Are you having an issue with your order or would you like to inquire about the status of your order with the number 678?

In [11]:
memories = support_agent.get_memories(user_id=customer_id)
for m in memories['results']:
    print(m)


{'id': '4a383650-8283-4723-a21c-2b6d1bde5356', 'memory': "Need help with recent order that hasn't arrived yet", 'hash': 'b4adcc05e9ecfaeb9a274db97448dd47', 'metadata': {'app_id': 'customer-support'}, 'created_at': '2024-12-19T02:14:54.096058-08:00', 'updated_at': None, 'user_id': 'jane_doe'}
{'id': 'b9aea8dd-a130-4652-985b-ce4f447a871f', 'memory': 'Order number is 678', 'hash': 'ca9fb46c98aa6f19b8b2ea922a7a9ac5', 'metadata': {'app_id': 'customer-support'}, 'created_at': '2024-12-19T02:17:33.856089-08:00', 'updated_at': '2024-12-19T02:21:22.527845-08:00', 'user_id': 'jane_doe'}
